In [1]:
import pandas as pd
import h5py
import os

In [28]:
# 4k Skims
working_dir_1 = r'L:\dev\esd\JF\sc_2040_esd_jf'
working_dir_2 = r'U:\soundcast_root\src\2040'
h5file = '2040-travelmodel.h5'

df_final = pd.DataFrame()

year = '2040'

tod = '7to8.h5'
h5_file_1 = h5py.File(os.path.join(working_dir_1,r'inputs\model\roster',tod))
source_1 = 'non-integrated'

h5_file_2 = h5py.File(os.path.join(working_dir_2,r'inputs\model\roster',tod))
source_2 = 'integrated'

matrix_list = ['svtl2t','h2tl2t','h3tl2t','svtl2c','h2tl2c','h3tl2c',]

for matrix_name in matrix_list:
    print matrix_name

    # Process Source 1 skims
    source = source_1
    _df_1 = pd.DataFrame(h5_file_1['Skims'][matrix_name][:])
    # Adjust index to start from zone 1 to 3700
    _df_1.index = _df_1.index+1
    _df_1.columns = _df_1.columns+1
    
    source = source_2
    _df_2 = pd.DataFrame(h5_file_2['Skims'][matrix_name][:])
    # Adjust index to start from zone 1 to 3700
    _df_2.index = _df_2.index+1
    _df_2.columns = _df_2.columns+1
    
    # Sum skim/trip values from origin
    _df_1 = pd.DataFrame(_df_1.unstack()).reset_index()
    _df_1 = _df_1.rename(columns={'level_0': 'otaz', 'level_1': 'dtaz', 0: 'trips'})

    # Get origin totals from otaz (too many records for full O-D comparison)
    df_1 = _df_1.groupby('otaz').sum()[['trips']]
    df_1 = df_1.reset_index()

    # Process Soundcast skims
    source = 'soundcast'

    df_2 = pd.DataFrame(_df_2.unstack()).reset_index()
    df_2 = df_2.rename(columns={'level_0': 'otaz', 'level_1': 'dtaz', 0: 'trips'})

    # Get total trips from otaz (too many records for full O-D comparison)
    df_2 = df_2.groupby('otaz').sum()[['trips']]
    df_2 = df_2.reset_index()

    # Merge results
    df = pd.merge(df_1,df_2,on='otaz', suffixes=['_1','_2'])
    df['matrix_name'] = matrix_name
    df['year'] = year

    df_final = df_final.append(df)

svtl2t
h2tl2t
h3tl2t
svtl2c
h2tl2c
h3tl2c


In [30]:
df_final.to_csv(r'R:\SoundCastDocuments\integrated\skim_comparison\2040\skim-compare.csv',
               index=False)